In [1]:
from coffea.nanoevents import NanoEventsFactory, BaseSchema
from coffea.analysis_tools import PackedSelection, Cutflow
import hist
from coffea import processor
from coffea.nanoevents.methods import vector
import awkward as ak
import dask_awkward as dak
import hist.dask as hda
import numba
import pandas as pd
import dask
import copy
import vector
from collections import namedtuple
vector.register_awkward()

/tmp/ipykernel_6147/2270838566.py:5: FutureWarning: In version 2024.7.0 (target date: 2024-06-30 11:59:59-05:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


In [2]:
events,report = NanoEventsFactory.from_root(
    '../data/p8_ee_ZH_ecm240/events_082532938.root:events',
    schemaclass=BaseSchema,
    metadata={'dataset':'ZH'},
    uproot_options={"allow_read_errors_with_report": True}
).events()

In [3]:
plot_props = pd.DataFrame({
    'Zm':{'name':'Zm','title':'Z Candidate mass','xlabel':'$Z_{mass}$ [GeV]','ylabel':'Events','bins':100,'xmin':0,'xmax':250},
    'Zm_zoom':{'name':'Zm_zoom','title':'Z Candidate mass','xlabel':'$Z_{mass}$ [GeV]','ylabel':'Events','bins':40,'xmin':80,'xmax':100},
    'Recoilm':{'name':'Recoilm','title':'Leptonic Recoil mass','xlabel':'$Recoil_{mass}$ [GeV]','ylabel':'Events','bins':100,'xmin':0,'xmax':200},
    'Recoilm_zoom':{'name':'Recoilm_zoom','title':'Leptonic Recoil mass','xlabel':'$Recoil_{mass}$ [GeV]','ylabel':'Events','bins':200,'xmin':80,'xmax':160},
    'Recoilm_zoom1':{'name':'Recoilm_zoom1','title':'Leptonic Recoil mass','xlabel':'$Recoil_{mass}$ [GeV]','ylabel':'Events','bins':100,'xmin':120,'xmax':140},
    'Recoilm_zoom2':{'name':'Recoilm_zoom2','title':'Leptonic Recoil mass','xlabel':'$Recoil_{mass}$ [GeV]','ylabel':'Events','bins':200,'xmin':120,'xmax':140},
    'Recoilm_zoom3':{'name':'Recoilm_zoom3','title':'Leptonic Recoil mass','xlabel':'$Recoil_{mass}$ [GeV]','ylabel':'Events','bins':400,'xmin':120,'xmax':140},
    'Recoilm_zoom4':{'name':'Recoilm_zoom4','title':'Leptonic Recoil mass','xlabel':'$Recoil_{mass}$ [GeV]','ylabel':'Events','bins':800,'xmin':120,'xmax':140},
    'Recoilm_zoom5':{'name':'Recoilm_zoom5','title':'Leptonic Recoil mass','xlabel':'$Recoil_{mass}$ [GeV]','ylabel':'Events','bins':2000,'xmin':120,'xmax':140},
    'Recoilm_zoom6':{'name':'Recoilm_zoom6','title':'Leptonic Recoil mass','xlabel':'$Recoil_{mass}$ [GeV]','ylabel':'Events','bins':100,'xmin':130.3,'xmax':140}
})
def get_1Dhist(name, var, flatten=True):
    '''
    name: eg. Zm
    var: eg. variable containing array of mass of Z
    flatten: If to flatten var before fill; True by default
    Returns a histogram
    '''
    props = plot_props[name]
    if flatten : var = dak.ravel(var)
    return hda.Hist.new.Reg(props.bins, props.xmin, props.xmax).Double().fill(var)

In [105]:
plot_props.columns.to_list()

['Zm',
 'Zm_zoom',
 'Recoilm',
 'Recoilm_zoom',
 'Recoilm_zoom1',
 'Recoilm_zoom2',
 'Recoilm_zoom3',
 'Recoilm_zoom4',
 'Recoilm_zoom5',
 'Recoilm_zoom6']

In [6]:
def get(events,collection,attribute,*cut):
    '''
    Get an attribute from a branch with or without a base cut.
    '''
    if len(cut) != 0:
        return events[collection+'/'+collection+'.'+attribute][cut[0]]
    return events[collection+'/'+collection+'.'+attribute]

def get_all(events,Collection,*basecut):
    '''
    Collect all the attributes of a collection into a namedtuple named particle, with or without a base cut
    '''
    prefix = '/'.join([Collection]*2)+'.'
    list_of_attr = [field.replace(prefix,'') for field in events.fields if field.startswith(prefix)]
    replace_list = ['.','[',']']
    valid_attr = list_of_attr
    for rep in replace_list:
        valid_attr = [field.replace(rep, '_') for field in valid_attr ]
    part = namedtuple('particle', valid_attr)
    return part(*[get(events,Collection,attr,*basecut) for attr in list_of_attr])

def get_reco(Reconstr_branch, needed_particle, events):
    '''
    Match the Reconstructed collection to the desired particle collection.
    '''
    part = namedtuple('particle', list(Reconstr_branch._fields))
    return part(*[getattr(Reconstr_branch,attr)[get(events,needed_particle,'index')] for attr in Reconstr_branch._fields])


In [18]:

#Create a Packed Selection object to get a cutflow later
cut = PackedSelection()
cut.add('No cut', dak.ones_like(dak.num(get(events,'ReconstructedParticles','energy'),axis=1),dtype=bool))

# Filter out any event with no reconstructed particles and generate Reconstructed Particle Attributes
#ak.mask preserves array length
at_least_one_recon = dak.num(get(events,'ReconstructedParticles','energy'), axis=1) > 0
good_events = dak.mask(events,at_least_one_recon)
cut.add('At least one Reco Particle', at_least_one_recon)

Reco = get_all(good_events,'ReconstructedParticles')
Muons = get_reco(Reco,'Muon#0',good_events)

# Create Array of Muon Lorentz Vector
Muon = ak.zip({"px":Muons.momentum_x,"py":Muons.momentum_y,"pz":Muons.momentum_z,"E":Muons.energy,"q":Muons.charge,}, with_name="Momentum4D")

# Muon pt > 10
Muon_pt_cut = dak.any(Muon.pt > 10.0, axis=1)
Muon = dak.mask(Muon, Muon_pt_cut) #ak.mask to preserve number of events
cut.add('Muon $p_T$ > 10 [GeV]',Muon_pt_cut)

# Construct Z candidates
# Produce all the combinations of Muon Pairs possible within an event
combs = dak.combinations(Muon,2)
# Get dimuons
mu1 , mu2 = dak.unzip(combs)
di_muon = mu1 + mu2
#Sort Dimuon with the dimuons which have a mass closer to Z mass

# Choose dimuon which is made up of two oppositely charged muons
q_sum_mask = dak.any((mu1.q + mu2.q) == 0, axis=1)
Z_cand = dak.mask(di_muon , q_sum_mask)
cut.add('Opp charge muons',q_sum_mask)


# Selection 0 : Only one Z candidate in an event
di_muon = dak.mask(di_muon, dak.num(di_muon) == 1)
cut.add('$N_Z = 1$',dak.num(Muon) == 2 ) #Having one Z candidate is same as having exactly two muons in an even

#Recoil Calculation
ecm = 240.0 #GeV # com energy
Recoil = ak.zip({"px":0.0-Z_cand.px,"py":0.0-Z_cand.py,"pz":0.0-Z_cand.pz,"E":ecm-Z_cand.E},with_name="Momentum4D")

sel0_ocl = cut.cutflow(*cut.names).yieldhist()

# Selection 1 : Selection 0 + 80 < M_Z < 100
zmassmask = (Z_cand.mass > 80) & (Z_cand.mass < 100)
Z_cand_sel1 = Z_cand[zmassmask]
Recoil_sel1 = Recoil[zmassmask]
zmassmask = ak.fill_none(zmassmask,[False],axis=0) #Replace None values at axis 0 with [False]
zmassmask = ak.flatten(zmassmask)
cut.add('80 < $M_Z$ < 100',zmassmask)

sel1_ocl = cut.cutflow(*cut.names).yieldhist()

#Prepare output
#Choose the required histograms and their assigned variables to fill
names = ['Zm','Zm_zoom','Recoilm','Recoilm_zoom','Recoilm_zoom1','Recoilm_zoom2','Recoilm_zoom3','Recoilm_zoom4','Recoilm_zoom5','Recoilm_zoom6']
vars_sel0 = ([Z_cand.mass]*2) + ([Recoil.mass]*8)
vars_sel1 = ([Z_cand_sel1.mass]*2) + ([Recoil_sel1.mass]*8)
Output = {
    'histograms': {
        'sel0':{name:get_1Dhist(name,var) for name,var in zip(names,vars_sel0)},
        'sel1':{name:get_1Dhist(name,var) for name,var in zip(names,vars_sel1)}
    },
    'cutflow': {
        'sel0': {'Onecut':sel0_ocl[0],'Cutflow':sel0_ocl[1],'Labels':sel0_ocl[2]},
        'sel1': {'Onecut':sel1_ocl[0],'Cutflow':sel1_ocl[1],'Labels':sel1_ocl[2]}
    }
}



In [19]:
(Output,) = dask.compute(Output)

In [20]:
Output

{'histograms': {'sel0': {'Zm': Hist(Regular(100, 0, 250, label='Axis 0'), storage=Double()) # Sum: 3036.0,
   'Zm_zoom': Hist(Regular(40, 80, 100, label='Axis 0'), storage=Double()) # Sum: 2324.0 (3036.0 with flow),
   'Recoilm': Hist(Regular(100, 0, 200, label='Axis 0'), storage=Double()) # Sum: 2920.0 (3036.0 with flow),
   'Recoilm_zoom': Hist(Regular(200, 80, 160, label='Axis 0'), storage=Double()) # Sum: 2647.0 (3036.0 with flow),
   'Recoilm_zoom1': Hist(Regular(100, 120, 140, label='Axis 0'), storage=Double()) # Sum: 2363.0 (3036.0 with flow),
   'Recoilm_zoom2': Hist(Regular(200, 120, 140, label='Axis 0'), storage=Double()) # Sum: 2363.0 (3036.0 with flow),
   'Recoilm_zoom3': Hist(Regular(400, 120, 140, label='Axis 0'), storage=Double()) # Sum: 2363.0 (3036.0 with flow),
   'Recoilm_zoom4': Hist(Regular(800, 120, 140, label='Axis 0'), storage=Double()) # Sum: 2363.0 (3036.0 with flow),
   'Recoilm_zoom5': Hist(Regular(2000, 120, 140, label='Axis 0'), storage=Double()) # Sum: 2

In [21]:
Output['cutflow']['sel1']['Cutflow'].values()

array([100000., 100000.,  99980.,   8681.,   3092.,   3036.,   2324.])

In [22]:
[100000., 100000.,  99980.,   8681.,   3092.,   3036.,   2324.]

[100000.0, 100000.0, 99980.0, 8681.0, 3092.0, 3036.0, 2324.0]

In [37]:
def clean(a):
    a = dak.fill_none(a,[],axis=0)
    return a[dak.num(a, axis=1) > 0]

In [98]:

def Reso_builder(lepton,resonance):
    '''
    Builds Resonance candidates with two oppositely charged leptons
    Input:    lepton(var*[var*LorentzVector]),
              resonance(float)
    Output: Reso(var*[var*LorentzVecctor]) sorted best to worst in axis 1
    '''
    #Create all the combinations
    combs = dak.combinations(lepton,2)
    # Get dileptons
    lep1 , lep2 = dak.unzip(combs)
    di_lep = lep1 + lep2
    # Choose dilep pair which is made up of two oppositely charged lep
    q_sum_mask = dak.any((lep1.q + lep2.q) == 0, axis=1)
    di_lep = dak.mask(di_lep , q_sum_mask)
    # Sort by closest mass to the resonance value
    sort_mask = dak.argsort(abs(resonance-di_muon.mass), axis=1)
    Reso = di_lep[sort_mask]
    return Reso

arg_ecm = 240.0 #GeV # com energy
arg_zmass = 91.0

#Create a Packed Selection object to get a cutflow later
cut = PackedSelection()
cut.add('No cut', dak.ones_like(dak.num(get(events,'ReconstructedParticles','energy'),axis=1),dtype=bool))

# Filter out any event with no reconstructed particles and generate Reconstructed Particle Attributes
#ak.mask preserves array length
at_least_one_recon = dak.num(get(events,'ReconstructedParticles','energy'), axis=1) > 0
good_events = dak.mask(events,at_least_one_recon)
cut.add('At least one Reco Particle', at_least_one_recon)

Reco = get_all(good_events,'ReconstructedParticles')
Muons = get_reco(Reco,'Muon#0',good_events)

# Create Array of Muon Lorentz Vector
Muon = ak.zip({"px":Muons.momentum_x,"py":Muons.momentum_y,"pz":Muons.momentum_z,"E":Muons.energy,"q":Muons.charge,}, with_name="Momentum4D")

# Muon pt > 10
Muon_pt_cut = dak.any(Muon.pt > 10.0, axis=1)
Muon = dak.mask(Muon, Muon_pt_cut) #ak.mask to preserve number of events
cut.add('Muon $p_T$ > 10 [GeV]',Muon_pt_cut)

Z_cand = Reso_builder(Muon, arg_zmass)

# Selection 0 : only one Z candidate
one_z = ak.num(sorted_di_muon,axis=1) == 1
Z_cand_sel0 = ak.mask(Z_cand, one_z)
cut.add('$N_Z = 1$', one_z ) 
sel0_ocl = cut.cutflow(*cut.names).yieldhist()

# Selection 1 : 80 < M_Z < 100
Z_mass_mask = (Z_cand_sel0.mass > 80.0) & (Z_cand_sel0.mass < 100.0)
Z_mass_mask = dak.fill_none(Z_mass_mask,[False],axis=0)
Z_mass_mask = dak.flatten(Z_mass_mask)
Z_cand_sel1 = ak.mask(Z_cand_sel0, Z_mass_mask)
cut.add('80 < $M_Z$ < 100',zmassmask)
sel1_ocl = cut.cutflow(*cut.names).yieldhist()

#Recoil Calculation
Recoil_sel0 = ak.zip({"px":0.0-Z_cand_sel0.px,"py":0.0-Z_cand_sel0.py,"pz":0.0-Z_cand_sel0.pz,"E":arg_ecm-Z_cand_sel0.E},with_name="Momentum4D")
Recoil_sel1 = ak.zip({"px":0.0-Z_cand_sel1.px,"py":0.0-Z_cand_sel1.py,"pz":0.0-Z_cand_sel1.pz,"E":arg_ecm-Z_cand_sel1.E},with_name="Momentum4D")

#Prepare output
#Choose the required histograms and their assigned variables to fill
names = ['Zm','Zm_zoom','Recoilm','Recoilm_zoom','Recoilm_zoom1','Recoilm_zoom2','Recoilm_zoom3','Recoilm_zoom4','Recoilm_zoom5','Recoilm_zoom6']
vars_sel0 = ([Z_cand.mass]*2) + ([Recoil.mass]*8)
vars_sel1 = ([Z_cand_sel1.mass]*2) + ([Recoil_sel1.mass]*8)
Output = {
    'histograms': {
        'sel0':{name:get_1Dhist(name,var) for name,var in zip(names,vars_sel0)},
        'sel1':{name:get_1Dhist(name,var) for name,var in zip(names,vars_sel1)}
    },
    'cutflow': {
        'sel0': {'Onecut':sel0_ocl[0],'Cutflow':sel0_ocl[1],'Labels':sel0_ocl[2]},
        'sel1': {'Onecut':sel1_ocl[0],'Cutflow':sel1_ocl[1],'Labels':sel1_ocl[2]}
    }
}

In [99]:
(Output,) = dask.compute(Output)

In [100]:
Output

{'histograms': {'sel0': {'Zm': Hist(Regular(100, 0, 250, label='Axis 0'), storage=Double()) # Sum: 3723.0,
   'Zm_zoom': Hist(Regular(40, 80, 100, label='Axis 0'), storage=Double()) # Sum: 2544.0 (3723.0 with flow),
   'Recoilm': Hist(Regular(100, 0, 200, label='Axis 0'), storage=Double()) # Sum: 2920.0 (3036.0 with flow),
   'Recoilm_zoom': Hist(Regular(200, 80, 160, label='Axis 0'), storage=Double()) # Sum: 2647.0 (3036.0 with flow),
   'Recoilm_zoom1': Hist(Regular(100, 120, 140, label='Axis 0'), storage=Double()) # Sum: 2363.0 (3036.0 with flow),
   'Recoilm_zoom2': Hist(Regular(200, 120, 140, label='Axis 0'), storage=Double()) # Sum: 2363.0 (3036.0 with flow),
   'Recoilm_zoom3': Hist(Regular(400, 120, 140, label='Axis 0'), storage=Double()) # Sum: 2363.0 (3036.0 with flow),
   'Recoilm_zoom4': Hist(Regular(800, 120, 140, label='Axis 0'), storage=Double()) # Sum: 2363.0 (3036.0 with flow),
   'Recoilm_zoom5': Hist(Regular(2000, 120, 140, label='Axis 0'), storage=Double()) # Sum: 2

In [101]:
Output['cutflow']['sel1']['Cutflow'].values()

array([100000., 100000.,  99980.,   8681.,   3036.,   2324.])